<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Computação com Xarray

Nesta lição, discutiremos como fazer cálculos científicos com objetos xarray. Nossos objetivos de aprendizagem são os seguintes. Ao final da aula, estaremos capaz de:

- Aplicar funções básicas de aritmética e numpy para `DataArrays` e `Dataset`;
- Usar as operações de redução com reconhecimento de mapeamento do Xarray (por exemplo, `média`,` soma`);
- Aplicar funções arbitrárias aos dados Xarray via `apply_ufunc`;
- Usar a propagação do Xarray para calcular em matrizes de dimensionalidades diferentes;
- Executar fluxos de trabalho "dividir / aplicar / combinar" no Xarray usando `groupby`, incluindo:
   - reduções dentro dos grupos;
   - propagações em grupos.
- Usar as funções `resample`,` rolling` e `coarsen` para manipular os dados.


In [ ]:
import expectexception
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

## Conjunto de dados de exemplo

Primeiro carregamos um conjunto de dados. Vamos usar o [NOAA Extended Reconstructed Sea Surface Temperature (ERSST) v5](https://www.ncdc.noaa.gov/data-access/marineocean-data/extended-reconstructed-sea-surface-temperature-ersst-v5), uma compilação em malha amplamente usada e confiável de dados históricos
voltando a 1854.

Uma vez que os dados são fornecidos por meio de um servidor [OPeNDAP](https://en.wikipedia.org/wiki/OPeNDAP), podemos carregá-lo diretamente,
sem baixar nada:


In [ ]:
### NOTA: Se centenas de pessoas se conectarem a este servidor ao mesmo tempo e baixarem o mesmo conjunto de dados,
###       as coisas podem ficar estranhas! Recomenda-se usar a cópia do Google Cloud.

# url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"
# # elimine uma variável desnecessária que complica algumas operações
# ds = xr.open_dataset(url, drop_variables=["time_bnds"])
# # levará um ou dois minutos para ser concluído
# ds = ds.sel(time=slice("1960", "2018")).load()
# ds

In [ ]:
import gcsfs

fs = gcsfs.GCSFileSystem(token="anon")
ds = xr.open_zarr(
    fs.get_mapper("gs://pangeo-noaa-ncei/noaa.ersst.v5.zarr"), consolidated=True
).load()
ds

Vamos fazer algumas visualizações básicas dos dados, apenas para ter certeza de que parecem
razoável.


In [ ]:
ds.sst[0].plot(vmin=-2, vmax=30)

## Aritmética Básica

`DataArrays` e conjuntos de dados Xarray funcionam perfeitamente com operadores aritméticos e
funções `numpy`.

Por exemplo, imagine que queremos converter a temperatura (dada em Celsius) para
Kelvin:


In [ ]:
sst_kelvin = ds.sst + 273.15
sst_kelvin

As dimensões e coordenadas foram preservadas após a operação.

<div class = "alert alert-warning">
     <strong> Aviso: </strong> embora muitos conjuntos de dados xarray tenham um atributo <code> units </code>, que é usado na plotagem,
     O Xarray não entende unidades inerentemente. No entanto, o trabalho está em andamento para integrar o xarray
     com <a href="https://pint.readthedocs.io/en/0.12/"> pint </a>, que fornecerá operações com reconhecimento total da unidade.
</div>

Podemos aplicar funções mais complexas, incluindo numpy `ufuncs`, para objetos Xarray.
Imagine que quiséssemos calcular a seguinte expressão como uma função de SST
($ \Theta $) em Kelvin:

$$ f(\Theta) =  0.5 \ln(\Theta^2) $$


In [ ]:
f = 0.5 * np.log(sst_kelvin ** 2)
f

## Aplicação de funções arbitrárias

É incrível podermos chamar `np.log(ds)` e fazer com que "simplesmente funcione". Porém, nem
todas as bibliotecas de terceiros funcionam dessa maneira.

Neste exemplo, usaremos funções do
[Gibbs Seawater Toolkit](https://teos-10.github.io/GSW-Python/), um pacote para
a termodinâmica da água do mar. Este pacote fornece `ufuncs` que operam em
matrizes numpy.


In [ ]:
import gsw

# an example function
# http://www.teos-10.org/pubs/gsw/html/gsw_t90_from_t68.html
gsw.t90_from_t68?

In [ ]:
gsw.t90_from_t68(ds.sst)  # -> returns a numpy array

Seria bom manter nossas dimensões e coordenadas. Nós podemos fazer isso
com `xr.apply_ufunc`:


In [ ]:
xr.apply_ufunc(gsw.t90_from_t68, ds.sst)

**Observação:** `apply_ufunc` é uma função poderosa e misteriosa.
Ela tem muitas opções para fazer coisas mais complicadas.
Infelizmente, não temos tempo para nos aprofundar aqui.
Consulte a [documentação Xarray](http://xarray.pydata.org/en/latest/generated/xarray.apply_ufunc.html) para obter mais detalhes.


## Reduções

Assim como no numpy, podemos reduzir DataArrays xarray ao longo de qualquer número de eixos:


In [ ]:
sst = ds.sst
sst.mean(axis=0)

In [ ]:
sst.mean(axis=(1, 2))

In [ ]:
sst.mean()

No entanto, em vez de realizar reduções nos eixos (como em numpy), podemos realizar
-las em dimensões. Isso acaba sendo uma grande conveniência, especialmente em
cálculos complexos quando você não consegue lembrar facilmente a qual eixo corresponde
qual dimensão:


In [ ]:
sst.mean(dim="time")

Todas as reduções numpy padrão (por exemplo, `min`,` max`, `sum`,` std`, etc.) são
acessível.


#### Exercício

Pegue a média de `sst` em longitude e latitude. Faça um gráfico simples contando a série temporal:

In [ ]:
# Seu código aqui

## Propagação

Propagação (*broadcasting*) se refere ao alinhamento de arranjos com diferentes números de
dimensões. As regras de propagação do Numpy, com base na forma do arranjo (`shape`), às vezes podem ser
difícil de entender e lembrar. Xarray faz *brodcasting* pelo nome de dimensão,
em vez do seu formato. Esta é uma grande conveniência.

Vamos agora criar dois arrays com algumas dimensões em comum. Para este exemplo, nós
vamos criar uma matriz de "pesos" proporcional ao cosseno de latitude.
Este é o fator de ponderação de área correto para dados em uma
malha lat-lon.

In [ ]:
weights = np.cos(np.deg2rad(ds.lat))
weights.dims

Se multiplicarmos por SST, "simplesmente funciona" e as matrizes são propagadas
devidamente:


In [ ]:
(ds.sst * weights).dims

<div class = "alert alert-warning">
     <strong> Cuidado: </strong> se as matrizes que estão sendo propagadas compartilham um nome de dimensão, mas têm coordenadas diferentes,
     elas serão alinhadas primeiro usando as configurações de alinhamento padrão do Xarray (incluindo o preenchimento dos valores ausentes com NaNs).
     Se não for isso que você deseja, é melhor chamar <code> align </code> explicitamente antes de transmitir.
</div>


## Reduções Ponderadas

Podemos imaginar o cálculo da média espacial ponderada do SST manualmente, assim:


In [ ]:
sst_mean = (ds.sst * weights).sum(dim=("lon", "lat")) / weights.sum(dim="lat")
sst_mean.plot()
plt.title("This is wrong!")

Isso estaria errado, no entanto, porque o denominador (`weights.sum(dim="lat")`)
precisa ser expandido para incluir a dimensão `lon` e modificado para levar em conta
os valores ausentes (pontos de terra).

Em geral, as reduções ponderadas em matrizes multidimensionais são complicadas. Para torná-las um pouco mais fáceis, o Xarray oferece um mecanismo para reduções ponderadas.
Fazemos isso criando um objeto intermediário `DataArray.weighted`, para
quais diferentes operações de redução podem ser aplicadas.


In [ ]:
sst_weighted = ds.sst.weighted(weights)
sst_weighted

In [ ]:
sst_weighted.mean(dim=("lon", "lat")).plot()
plt.title("Correct Global Mean SST")

## Agrupar por - *groupby*

O Xarray copia a funcionalidade de agrupamento muito útil do Pandas, permitindo o fluxo de trabalho "divide /
aplica / combina" em DataArrays e Dataset xarray.

Para fornecer um exemplo motivado fisicamente, vamos examinar uma série temporal de SST em
um único ponto.


In [ ]:
ds.sst.sel(lon=300, lat=50).plot()

Como podemos ver no gráfico, a série do tempo em qualquer ponto é totalmente
dominada pelo ciclo sazonal. Gostaríamos de remover esse ciclo sazonal
(chamada de "climatologia"), a fim de ver melhor as variações de longo prazo em
temperatura. Podemos fazer isso usando **groupby**.

Antes de avançar, observamos que o xarray analisou corretamente o índice de tempo,
resultando em um índice de data e hora do Pandas na dimensão do tempo:


In [ ]:
ds.time

A sintaxe do *groupby* de Xarray é quase idêntica à do Pandas:


In [ ]:
ds.groupby?

### Etapa de divisão

O argumento mais importante é `grup`: ele define os valores únicos que usaremos para "dividir" os dados para análise agrupada. Podemos passar um DataArray ou um nome de uma variável no conjunto de dados. Vamos primeiro usar um DataArray. Assim como com Pandas, podemos usar o índice de tempo para extrair componentes específicos de datas e
horários. Xarray usa uma sintaxe especial para este `.dt`, chamada de `DatetimeAccessor`:


In [ ]:
ds.time.dt

In [ ]:
ds.time.dt.month

Podemos usar esses arrays em uma operação *groupby*:


In [ ]:
gb = ds.groupby(ds.time.dt.month)
gb

Xarray também oferece uma sintaxe mais concisa quando a variável que você está agrupando já está presente no conjunto de dados. Isso é idêntico à linha anterior:


In [ ]:
gb = ds.groupby("time.month")
gb

Agora que os dados estão divididos, podemos iterar manualmente no grupo. O
iterador retorna a chave (nome do grupo) e o valor (o conjunto de dados real
correspondente a esse grupo) para cada grupo:


In [ ]:
for group_name, group_ds in gb:
    # stop iterating after the first loop
    break
print(group_name)
group_ds

### Aplicar e combinar

Agora que temos grupos definidos, é hora de "aplicar" um cálculo ao
grupo. Como no Pandas, esses cálculos podem ser:

- *Agregação*: reduz o tamanho do grupo;
- *Transformação*: preserva o tamanho total do grupo.

No final da etapa de aplicação, o xarray combinará automaticamente os grupos agregados/transformados de volta em um único objeto.


#### Agregações


Como no Pandas, o objeto *groupby* do xarray tem muitas operações de agregação integradas
(por exemplo, `mean`,` min`, `max`,` std`, etc):


In [ ]:
ds_mm = gb.mean(dim="time")
ds_mm

Então, fizemos o que queríamos fazer: calcular a climatologia em cada ponto do
conjunto de dados. Vamos dar uma pequena olhada nos dados.

_Climatologia em um ponto específico do Atlântico Norte_


In [ ]:
ds_mm.sst.sel(lon=300, lat=50).plot()

_Zonal Mean Climatolgoy_


In [ ]:
ds_mm.sst.mean(dim="lon").plot.contourf(x="month", levels=12, vmin=-2, vmax=30)

_Diferença na Climatologia entre Janeiro e Julho_


In [ ]:
(ds_mm.sst.sel(month=1) - ds_mm.sst.sel(month=7)).plot(vmax=10)

#### Transformações

Agora queremos _remover_ esta climatologia do conjunto de dados, para examinar o
residual, chamado de _anomalia_, que é a parte interessante de uma perspectiva climática.
A remoção da climatologia sazonal é um exemplo perfeito de uma
transformação: opera sobre um grupo, mas não altera o tamanho do
conjunto de dados. Aqui está uma maneira de codificá-lo:


In [ ]:
def remove_time_mean(x):
    return x - x.mean(dim="time")


ds_anom = ds.groupby("time.month").map(remove_time_mean)
ds_anom

O Xarray facilita esses tipos de transformações, oferecendo suporte a *groupby* aritmético. Esse conceito é explicado mais facilmente com um exemplo:


In [ ]:
gb = ds.groupby("time.month")
ds_anom = gb - gb.mean(dim="time")
ds_anom

Agora podemos ver o sinal do clima sem a influência esmagadora do
ciclo sazonal.

_Série temporal em um único ponto no Atlântico Norte_

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot()

_Diferença entre 1º de janeiro de 2018 e 1º de janeiro de 1960_


In [ ]:
(ds_anom.sel(time="2018-01-01") - ds_anom.sel(time="1960-01-01")).sst.plot()

## Relacionado ao *Grouby*: *Resample*, *Rolling*, *Coarsen*

*Resample* em xarray é quase idêntico ao Pandas. É efetivamente uma operação *groupby* e usa a mesma sintaxe básica. Pode ser aplicado apenas ao índice das dimensões temporais. Aqui, calculamos a média de cinco anos:

In [ ]:
resample_obj = ds_anom.resample(time="5Y")
resample_obj

In [ ]:
ds_anom_resample = resample_obj.mean(dim="time")
ds_anom_resample

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot()
ds_anom_resample.sst.sel(lon=300, lat=50).plot(marker="o")

<div class="alert alert-info">
    <strong>Nota:</strong> <code>resample</code> Funciona apenas com índices de séries temporais.
</div>

*Rolling* também é semelhante ao pandas, mas pode ser aplicado em qualquer dimensão. A função
funciona com coordenadas lógicas.


In [ ]:
ds_anom_rolling = ds_anom.rolling(time=12, center=True).mean()
ds_anom_rolling

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot(label="monthly anom")
ds_anom_resample.sst.sel(lon=300, lat=50).plot(
    marker="o", label="5 year resample"
)
ds_anom_rolling.sst.sel(lon=300, lat=50).plot(label="12 month rolling mean")
plt.legend()

`coarsen` faz algo semelhante a `resample`, mas sem estar ciente do tempo.
Ele opera em coordenadas lógicas apenas, mas pode trabalhar em várias dimensões por vez.


In [ ]:
ds_anom_coarsen_time = ds_anom.coarsen(time=12).mean()

ds_anom_rolling.sst.sel(lon=300, lat=50).plot(label="12 month rolling mean")
ds_anom_coarsen_time.sst.sel(lon=300, lat=50).plot(
    marker="^", label="12 item coarsen"
)
plt.legend()

In [ ]:
%%expect_exception
ds_anom_coarsen_space = ds_anom.coarsen(lon=4, lat=4).mean()

In [ ]:
ds_anom_coarsen_space = (
    ds_anom.isel(lat=slice(0, -1)).coarsen(lon=4, lat=4).mean()
)
ds_anom_coarsen_space

In [ ]:
ds_anom_coarsen_space.sst.isel(time=0).plot()